In [3]:
from bardapi import Bard
from typing import Any, List, Mapping, Optional
from getpass import getpass
import os

In [11]:
from langchain.chains import RetrievalQA #-> for retrieval QA
from langchain.chains.question_answering import load_qa_chain #-> for load QA
from langchain.indexes import VectorstoreIndexCreator #-> for retrieval QA using vectors index creator. it uses fewer codes with less tokens. So I think this is the best way.

from langchain.document_loaders import TextLoader, PyPDFLoader, DataFrameLoader #-> types of data loader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter #-> to splits data to chunks
from langchain.vectorstores import Chroma #to create a db of chunks
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

In [5]:
os.environ['_BARD_API_KEY'] = "dwjgofllW12z8zTkVaT-GrkUJvxnb0I4C2HRMnhzzYkiX75RbT9sHfiXzxlxqhgJ8zlFhA."

In [6]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
class BardLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        response = Bard(token=os.environ['_BARD_API_KEY']).get_answer(prompt)['content']
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

In [7]:
llm=BardLLM()

In [12]:
embeddings = SentenceTransformerEmbeddings(model_name="keepitreal/vietnamese-sbert")

# embeddings = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

In [13]:
db = Chroma(persist_directory="./chroma_index_2", embedding_function=embeddings)

In [14]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k":10}),
    return_source_documents=True)

In [9]:
from langchain import PromptTemplate

In [15]:
question_template = """
  Use the following pieces of context to answer the question at the end.
  If you don't know the answer, just say that you don't know, don't try to
  make up an answer.
  Imagine you're talking to a friend and use natural language and phrasing.
  You can only use Vietnamese and do not use other languages.

  {context}
  Question: {question}
  Helpful Answer:
  """
QUESTION_PROMPT = PromptTemplate(
        template=question_template, input_variables=["question", "context"]
    )

In [16]:
question = "Gợi ý cho tôi những bộ phim tương tự Toy Story"
# question = str(input())
qa.combine_documents_chain.llm_chain.prompt = QUESTION_PROMPT
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True

In [17]:
result = qa({"query":question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [18]:
print(result['result'])

**Gợi ý phim tương tự Toy Story**

Toy Story là một bộ phim hoạt hình kinh điển của Pixar, kể về câu chuyện của những món đồ chơi có ý thức và có thể hoạt động khi không có con người. Phim đã được khen ngợi về hình ảnh, âm nhạc, cốt truyện và nhân vật.

Dựa trên những tiêu chí này, mình có thể gợi ý cho bạn một số bộ phim tương tự Toy Story như sau:

* **A Bug's Life (1998)**: Phim kể về cuộc chiến giữa một đàn kiến và một nhóm bọ cánh cứng ăn thịt. Phim có hình ảnh đẹp mắt, cốt truyện hài hước và nhân vật đáng yêu.
* **Monsters, Inc. (2001)**: Phim kể về hai con quái vật phải làm việc cho một công ty chuyên thu thập tiếng thét của trẻ em. Phim có hình ảnh sáng tạo, cốt truyện hấp dẫn và nhân vật thú vị.
* **Finding Nemo (2003)**: Phim kể về cuộc hành trình của một chú cá hề đi tìm con trai bị lạc. Phim có hình ảnh sống động, cốt truyện cảm động và nhân vật dễ thương.
* **Wall-E (2008)**: Phim kể về một chú robot dọn rác sống sót trong một thế giới bị bỏ hoang. Phim có hình ảnh đẹp mắt